In [540]:
import requests
import pandas as pd
import os
import pprint
import geopandas
import matplotlib.pyplot as plt
import geodatasets
import contextily as cx
import numpy as np
import json
import googlemaps
from googlemaps.places import find_place
import geopy
import bisect 
from math import radians, cos, sin, asin, sqrt

API_KEY = 'MY API KEY'



## First attempt at geocoding

In [260]:
condos = pd.read_csv("condo_projects.csv", keep_default_na=False).drop_duplicates()
condos['lat'] = ''
condos['lng'] = ''

for i in range(8618,8620): #range(0,len(condos)):
    name_address = condos['project name'][i] + ', ' + condos['address'][i] + ', '+ condos['location'][i] + ', Ontario, Canada'
    address =  condos['address'][i] + ', '+ condos['location'][i] + ', Ontario, Canada'
    if len(info.json()['results'])==1:
    info = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+name_address+'&key='+API_KEY)
        condos['lat'][i] = info.json()['results'][0]['geometry']['location']['lat']
        condos['lng'][i] = info.json()['results'][0]['geometry']['location']['lng']
    
    #sometimes it finds multiple addresses. in these cases usually it is the project name that is confusing, so we search without it
    else:  
        info2 = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+address+'&key='+API_KEY)
        if len(info2.json()['results'])==1:
            condos['lat'][i] = info2.json()['results'][0]['geometry']['location']['lat']
            condos['lng'][i] = info2.json()['results'][0]['geometry']['location']['lng']
     
    #if still multiple or no addresses are found, give up
        else:
            condos['lat'][i] = 'more than one'
            condos['lng'][i] = 'more than one'
            
condos.to_csv("condo_projects_geocoded.csv" , index=False)

## second attempt at geocoding:
   
fill information for condos for which I have more than one address or none, designated by 'more than one' in lat lng column

In [361]:

condos = pd.read_csv("condo_projects_geocoded.csv")

client = googlemaps.Client(key=API_KEY)

for i in range(0,len(condos)):
    if condos['lat'][i] == 'more than one':
        name_address = str(condos['project name'][i]) + ', ' + condos['address'][i] + ', '+ condos['location'][i] + ', Ontario, Canada'
        address =  condos['address'][i] + ', '+ condos['location'][i] + ', Ontario, Canada'
        if client.places_autocomplete(name_address)!=[]:
            info = client.geocode(client.places_autocomplete(name_address)[0])[0]
            condos['lat'][i] = info['geometry']['location']['lat']
            condos['lng'][i] = info['geometry']['location']['lng']
        elif client.places_autocomplete(address)!=[]:
            info = client.geocode(client.places_autocomplete(address)[0])[0]
            condos['lat'][i] = info['geometry']['location']['lat']
            condos['lng'][i] = info['geometry']['location']['lng']
        elif client.addressvalidation(address)!=[]:
            info = client.addressvalidation(address)
            condos['lat'][i] = info['result']['geocode']['location']['latitude']
            condos['lng'][i] = info['result']['geocode']['location']['longitude']
            
condos.to_csv("condo_projects_geocoded.csv" , index=False)

## Third attempt at geocoding:
correct geocodes that fall outside Ontario, which means they are geocoded by mistake


In [452]:

condos = pd.read_csv("condo_projects_geocoded.csv")

cols = ['lat','lng','number of units']
condos[cols] = condos[cols].apply(pd.to_numeric, errors='coerce')

#if point is outside Ontario bounds:
[ xmin, ymin, xmax, ymax ] = [-95.16, 41.66, -74.34, 56.86]
conditions = [
    (condos['lat']<ymin) ,
    (condos['lat']>ymax) , 
    (condos['lng']<xmin) , 
    (condos['lng']>xmax)
]
choices = [1,1,1,1]
condos['wrong_coords'] = np.select(conditions, choices, default=0)



for i in range(0,len(condos)):
    if condos['wrong_coords'][i] == 1:
        address =  condos['address'][i] + ', '+ condos['location'][i] + ', Ontario, Canada'
        info = client.addressvalidation(address)
        condos['lat'][i] = info['result']['geocode']['location']['latitude']
        condos['lng'][i] = info['result']['geocode']['location']['longitude']
                            
condos.to_csv("condo_projects_geocoded.csv" , index=False)

/var/folders/t6/g6kz7nts2wdcdc7nw7g1r1vm0000gn/T/ipykernel_22814/468888412.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condos['lat'][i] = info['result']['geocode']['location']['latitude']
/var/folders/t6/g6kz7nts2wdcdc7nw7g1r1vm0000gn/T/ipykernel_22814/468888412.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condos['lng'][i] = info['result']['geocode']['location']['longitude']


ADDRESS:   6061&6079 Rutherford Rd., Vaughan, Ontario, Canada
FORMATTED ADDRESS:    6061&6079 Rutherford Road, Vaughan, ON, Canada
ADDRESS:   253 & 255 Lake Driveway West, Ajax, Ontario, Canada
FORMATTED ADDRESS:    253 & 255 Lake Driveway West, Ajax, ON L1S 5B5, Canada
ADDRESS:   Hazelton & Yorkville Ave., Toronto, Ontario, Canada
FORMATTED ADDRESS:    Hazelton &, Yorkville Avenue, Toronto, ON M5R 1B9, Canada
ADDRESS:   550 Steddick Court, Mississauga, Ontario, Canada
FORMATTED ADDRESS:    550 Steddick Court, Mississauga, ON L5R 3S8, Canada
ADDRESS:   NE Corner Of Salem & Harper Hill Rd., Ajax, Ontario, Canada
FORMATTED ADDRESS:    NE Corner Of Salem &, Harper-Hill Drive, Ajax, ON L1Z 0R3, Canada
ADDRESS:   NE Corner Of Salem & Harper Hill Rd., Ajax, Ontario, Canada
FORMATTED ADDRESS:    NE Corner Of Salem &, Harper-Hill Drive, Ajax, ON L1Z 0R3, Canada
ADDRESS:   57 & 59 East Liberty St., Toronto, Ontario, Canada
FORMATTED ADDRESS:    57 & 59 East Liberty Street, Toronto, ON, Canada
A

ADDRESS:   386-394 Symington Av.,485 Perth Av. &17 Kingsley Av., Toronto, Ontario, Canada
FORMATTED ADDRESS:    394 Symington Av #386, &17, ON, Canada


## Nearby condos
for each condo building, find the minimum distance one needs to go to find at least 3,5,10 other available condos from different parent companies.

In [709]:
# read GTA data first:
condos = pd.read_csv("condo_projects_geocoded.csv")

epsg = '3857'  #mapping projection identifier

cols = ['lat','lng','number of units']
condos[cols] = condos[cols].apply(pd.to_numeric, errors='coerce')


condos_geo = geopandas.GeoDataFrame(
    condos, geometry=geopandas.points_from_xy(pd.to_numeric(condos.lng, errors='coerce').astype(float), pd.to_numeric(condos.lat, errors='coerce').astype(float)), crs="EPSG:"+epsg
)

condos_gta = condos_geo.loc[(condos_geo['lat']>43.3) & 
                            (condos_geo['lat']<44.5) & 
                            (condos_geo['lng']>-80) & 
                            (condos_geo['lng']<-79)].reset_index()

In [710]:
#vectorized geo-distance calculation
def haversine(lat1, lng1, lat2, lng2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    lat1_rad = lat1*np.pi/180.0
    lng1_rad = lng1*np.pi/180.0
    lat2_rad = lat2*np.pi/180.0
    lng2_rad = lng2*np.pi/180.0
    
    dlat = lat1_rad - lat2_rad
    dlng = lng1_rad - lng2_rad
    
    # haversine formula 
    a = np.square(np.sin(np.divide(dlat, 2)) ) + np.cos(lat1_rad) * np.cos(lat2_rad) * np.square(np.sin(np.divide(dlng, 2)) )
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r

N_closest = 101

parent_companies = pd.read_csv("parent_crosswalk.csv")
condos_gta = condos_gta.merge(parent_companies,  how='left', on='link')


condos_gta['competitor_distance_list'] = None
for i in range(0,len(condos_gta)):
    
    dist = haversine(condos_gta.loc[i].lat, condos_gta.loc[i].lng,  condos_gta.lat, condos_gta.lng)
    
    neighbors_df = pd.DataFrame(
        {'dist': list(dist),
         'parent_ID': list(condos_gta.parent_ID),
         'N_under_1km': 0,
         'N_under_2km': 0
        }
    ).groupby('parent_ID').min().reset_index()  #find closest of each company's apartments to apartment i
    
    condos_gta.at[i,'competitor_distance_list'] = sorted(neighbors_df.dist)[0:N_closest]
    condos_gta.at[i,'distance_to_5th'] = condos_gta['competitor_distance_list'][i][5]
    condos_gta.at[i,'distance_to_10th'] = condos_gta['competitor_distance_list'][i][10]
    condos_gta.at[i,'distance_to_20th'] = condos_gta['competitor_distance_list'][i][20]
    condos_gta.at[i,'distance_to_50th'] = condos_gta['competitor_distance_list'][i][50]
    condos_gta.at[i,'distance_to_100th'] = condos_gta['competitor_distance_list'][i][100]
    
    condos_gta.at[i,'N_under_1km'] = neighbors_df['dist'][neighbors_df['dist']<1].count()
    condos_gta.at[i,'N_under_2km'] = neighbors_df['dist'][neighbors_df['dist']<2].count()
condos_gta.to_csv("condo_projects_geocoded_GTA.csv" , index=False)
